# WILDTRACK

## Data description

Data from object detection of video of pedestrians walking by an entry of a building, there is a total of 20 seconds of video annoted (2 frames per second) and the video is filmed with different cameras, so there are annotation for each of these videos. Data is available at the following [link](https://www.epfl.ch/labs/cvlab/data/data-wildtrack/).

Add the files in the folder `data/wildtrack`.

In [1]:
import pandas as pd
import json
from PIL import Image

## Preprocessing json file

The annotations are available in json format, we will use the annotations for the third camera, for each frame of the video we save all the bouding boxes annotated and we will have the information: x-position, y-position, width, area, time (frame number), points of the 4 corners.

In [2]:
df = {"object": [], "frame" : [], "xmin": [], "xmax": [], "ymin": [], "ymax": [], "points": []}
for i in range(0, 2000, 5):
    i_s = ("00000000" + str(i))[-8:]
    data = json.load(open("../data/wildtrack/annotations_positions/" + i_s + ".json", 'r'))
    for d in data:
        df['object'].append(d['personID'])
        df['frame'].append(i)
        xmin = max(0, d['views'][2]['xmin'])
        xmax = min(1920, d['views'][2]['xmax'])
        ymin = max(0, d['views'][2]['ymin'])
        ymax = min(1280, d['views'][2]['ymax'])
        df['xmin'].append(xmin)
        df['xmax'].append(xmax)
        df['ymin'].append(ymin)
        df['ymax'].append(ymax)

        points = []
        corners = [
                ["xmin", "ymin"],
                ["xmax", "ymin"],
                ["xmax", "ymax"],
                ["xmin", "ymax"],
                ["xmin", "ymin"],
            ]
        for corner in corners:
            points.append([df[corner[0]][-1], df[corner[1]][-1]])
        df["points"].append(points)
df = pd.DataFrame(df)
df = df[((df.xmin != -1) & (df.xmax != -1) & (df.ymin != -1) & (df.ymax != -1))]
df["timestep"] = (df.frame - df.frame.min())//5
df['area'] = (df.xmax - df.xmin) * (df.ymax - df.ymin)
df['xcenter'] = (df.xmax + df.xmin)/2
df['ycenter'] = (df.ymax + df.ymin)/2
df.to_csv("../data/processed/wildtrack.csv", index = False)

## Reduce image resolution

We will reduce the resolution from 1920x1280 to a smaller resolution of 384x288 so it can be easily shown in the web application.

In [3]:
for i in range(0, 2000, 5):
    i_s = str(i)
    i_s = "00000000" + i_s
    i_s = i_s[-8:]
    img = Image.open("../data/wildtrack/Image_subsets/C3/" + i_s + ".png")
    img = img.resize((384, 288), Image.ANTIALIAS)
    img.save("../app/static/data/c3_small/" + i_s + ".png")